In [4]:

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display_html
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
seed=1
from sklearn.metrics import accuracy_score

In [5]:
ds_train= pd.read_csv('data/train.csv')
ds_train=shuffle(ds_train,random_state=0)
ds_test= pd.read_csv('data/test.csv')

df=pd.DataFrame(ds_train)
print(df.head())

train_labels=ds_train["cancelled"]
ds_train=ds_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders',
                            'delivered_time','cancelled','order_id'],axis=1)

ds_train.to_numpy()
data_train=np.nan_to_num(ds_train)
train_labels.to_numpy()
train_examples=data_train

X = train_examples
y = train_labels

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/1000)

ds_train.describe().transpose()


                 order_time  order_id           order_date  \
264355  2021-02-01 16:12:15    349489  2021-02-01 00:00:00   
350939  2021-02-03 17:00:26    275043  2021-02-03 00:00:00   
287917  2021-02-02 13:43:54    294344  2021-02-02 00:00:00   
422901  2021-02-05 15:28:44    183847  2021-02-05 00:00:00   
289800  2021-02-02 14:01:00    296226  2021-02-02 00:00:00   

                 allot_time          accept_time          pickup_time  \
264355  2021-02-01 16:12:47  2021-02-01 16:13:38  2021-02-01 16:24:37   
350939  2021-02-03 17:00:28  2021-02-03 17:00:35  2021-02-03 17:07:32   
287917  2021-02-02 13:44:10  2021-02-02 13:47:25  2021-02-02 13:54:18   
422901  2021-02-05 15:30:41  2021-02-05 15:32:55  2021-02-05 15:42:10   
289800  2021-02-02 14:01:01  2021-02-02 14:01:20                  NaN   

             delivered_time  rider_id  first_mile_distance  \
264355  2021-02-01 16:43:02      8092               1.0442   
350939  2021-02-03 17:23:38      7047               1.2496   
28

,count,mean,std,min,25%,50%,75%,max
first_mile_distance,450000.0,1.229889,0.846183,0.000134,0.539575,1.1387,1.853000,42.038100
last_mile_distance,450000.0,2.968873,1.884124,0.000000,1.470000,2.6700,4.220000,22.410000
alloted_orders,433052.0,104.620909,90.135492,1.000000,36.000000,81.0000,147.000000,567.000000
delivered_orders,432659.0,103.950448,89.639646,1.000000,36.000000,81.0000,146.000000,562.000000
lifetime_order_count,449947.0,853.640664,1502.976162,0.000000,165.000000,396.0000,948.000000,30469.000000
session_time,446325.0,220.474779,176.713853,0.000000,84.100000,175.5500,316.766667,1298.966667


In [6]:

def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(3,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  opt = tf.keras.optimizers.SGD(lr=0.002)
  model.compile(optimizer=opt,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])         
  return model


In [7]:
from numpy.core.numeric import NaN
estimator = KerasRegressor(build_fn=get_basic_model, nb_epoch=100, batch_size=100, verbose=False)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

estimator.fit(X, y)
prediction = estimator.predict(X)
np.any(np.isnan(prediction))
np.all(np.isfinite(prediction))
accuracy_score(y, prediction)

2022-02-04 23:31:52.017653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 23:32:05.023575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 23:32:05.939423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 23:32:18.708611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 23:32:19.615583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 23:32:32.556390: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 23:32:33.559751: I tensorflow/core/grappler/optimizers/cust

Results: -0.07 (0.00) MSE


2022-02-04 23:34:08.137310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 23:34:21.860715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=4,verbose=1, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid))

model.evaluate(X_test,y_test)

In [ ]:
test=pd.read_csv("/content/test.csv")
test_id=test["order_id"]
test=test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','undelivered_orders',
                            'order_id'],axis=1)
test.to_numpy()
test=np.nan_to_num(test)

ans=model.predict(test)

sum(ans)/len(ans)

data=pd.DataFrame()
ans=np.where(ans > 0.5, 1, 0)
data.insert(0,"order_id",test_id)
data.insert(1,"cancelled",ans)

data.head()

data.to_csv("/content/model2.csv",index=False)

